# Проект по SQL

**Описание проекта**   
Наша компания приобрела крупный сервис для чтения книг по подписке. Необходимо проанализировать базу данных, в которой содержится информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.  
Эти данные помогут сформулировать ценностное предложение для нового продукта.  
Данные содержатся таблицах:  
books - содержит данные о книгах;  
authors - содержит данные об авторах;  
publishers - содержит данные об издательствах;  
ratings - содержит данные о пользовательских оценках книг;  
reviews - содержит данные о пользовательских обзорах на книги.

**Содержание проекта**  
1. [Открытие файлов и изучение информации в таблицах](#review1)
2. [Запросы к базе данных](#query2)
    * [Подсчет количества книг, вышедших после 1 января 2000 года](#query21)
    * [Подсчет количества обзоров и средней оценки для каждой книги](#query22)
    * [Определение издательства, которое выпустило наибольшее число книг толще 50 страниц](#query23)
    * [Определение автора с самой высокой средней оценкой книг, у которых имеется не менее 50 оценок](#query24)
    * [Подсчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок](#query25)
3. [Выводы](#conclusion3)

# Открытие файлов и изучение информации в таблицах <a id="review1"></a>

In [1]:
# импортируем нужные для работы библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# выведем первые 5 строк таблицы books
query = '''
SELECT *
FROM books
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица `books` cодержит данные о книгах:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [3]:
# выведем первые 5 строк таблицы authors
query = '''
SELECT *
FROM authors
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица `authors` cодержит данные об авторах:
- `author_id` — идентификатор автора;
- `author` — имя автора.

In [4]:
# выведем первые 5 строк таблицы publishers
query = '''
SELECT *
FROM publishers
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица `publishers` cодержит данные об издательствах:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [5]:
# выведем первые 5 строк таблицы ratings
query = '''
SELECT *
FROM ratings
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица `ratings` cодержит данные о пользовательских оценках книг:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [6]:
# выведем первые 5 строк таблицы reviews
query = '''
SELECT *
FROM reviews
LIMIT 5
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Таблица `reviews` cодержит данные о пользовательских обзорах на книги:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

# Запросы к базе данных <a id="query2"></a>

## Подсчет количества книг, вышедших после 1 января 2000 года <a id="query21"></a>

In [7]:
query = """
SELECT COUNT(book_id)
FROM books
WHERE CAST(publication_date AS timestamp) > '2000-01-01'
"""
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Количество книг, вышедших после 01.01.2000 г., составляет 819.

## Подсчет количества обзоров и средней оценки для каждой книги <a id="query22"></a>

In [8]:
query = """
SELECT title,
       COUNT(DISTINCT(review_id)) AS review_count,
       ROUND(AVG(rating), 1) AS rating_avg
FROM books AS b 
     LEFT JOIN ratings AS r ON b.book_id = r.book_id
     LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
GROUP BY b.book_id
ORDER BY rating_avg DESC, review_count DESC
"""
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,title,review_count,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.0
1,School's Out—Forever (Maximum Ride #2),3,5.0
2,Moneyball: The Art of Winning an Unfair Game,3,5.0
3,Crucial Conversations: Tools for Talking When ...,2,5.0
4,Misty of Chincoteague (Misty #1),2,5.0
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.3
996,Drowning Ruth,3,2.0
997,His Excellency: George Washington,2,2.0
998,Junky,2,2.0


Общее количество книг составляет 1000. Книга с наибольшим количеством обзоров и рейтингом - A Dirty Job (Grim Reaper #1)

## Определение издательства, которое выпустило наибольшее число книг толще 50 страниц <a id="query23"></a>

In [9]:
query = """
--определяем кол-во книг, выпущенное каждым издательством
WITH publisher_books_count AS (
     SELECT publisher,
            COUNT(book_id) AS books_count
     FROM publishers AS p
     INNER JOIN books AS b 
     ON p.publisher_id = b.publisher_id
     WHERE num_pages > 50
     GROUP BY publisher
     ORDER BY books_count DESC)

-- определяем издательство с максимальным количеством выпущенных книг
SELECT *
FROM publisher_books_count
WHERE books_count = (SELECT MAX(books_count)
                     FROM publisher_books_count)                   
"""
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,books_count
0,Penguin Books,42


Издательством, которое выпустило наибольшее число книг толще 50 страниц, является Penguin Books (выпустило 42 книги).

## Определение автора с самой высокой средней оценкой книг, у которых имеется не менее 50 оценок <a id="query24"></a>

In [10]:
query = """
SELECT author,
       ROUND(AVG(rating), 1) AS rating_mean
FROM authors AS a
     JOIN books AS b 
     ON a.author_id = b.author_id
     JOIN ratings AS r 
     ON b.book_id = r.book_id
WHERE b.book_id IN (
      SELECT book_id
      FROM ratings 
      GROUP BY book_id
      HAVING COUNT(rating_id) >= 50
      ORDER BY COUNT(rating_id) DESC)

GROUP BY author
ORDER BY rating_mean DESC
LIMIT 1
"""
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating_mean
0,J.K. Rowling/Mary GrandPré,4.3


Автором с самой высокой средней оценкой книг, у которых имеется не менее 50 оценок, является J.K. Rowling/Mary GrandPré (рейтинг 4,3).

## Подсчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок <a id="query25"></a>

In [11]:
query = """
WITH reviews_users AS (
     SELECT username, 
            COUNT(review_id) AS review_count
     FROM reviews 
     WHERE username IN (
     --определяем пользователей, которые поставили больше 48 оценок
                       SELECT username
                       FROM ratings
                       GROUP BY username
                       HAVING COUNT(rating_id) > 48)
     GROUP BY  username)
SELECT ROUND(AVG(review_count), 0)
FROM reviews_users   
"""
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,round
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24.

# Выводы <a id="conclusion3"></a>

По итогам проведения анализа базы данных можно сделать следующие выводы:  
• Количество книг, вышедших после 01.01.2000 г., составляет 819.  
• Общее количество книг составляет 1000. Книга с наибольшим количеством обзоров и рейтингом - A Dirty Job (Grim Reaper #1).  
• Издательством, которое выпустило наибольшее число книг толще 50 страниц, является Penguin Books (выпустило 42 книги).  
• Автором с самой высокой средней оценкой книг, у которых имеется не менее 50 оценок, является J.K. Rowling/Mary GrandPré (рейтинг 4,3).  
• Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24.